In [1]:
# Additional Feature Engineering Steps - Fix Feature Dominance
# Run this AFTER your current feature engineering completes

import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from sklearn.preprocessing import RobustScaler, StandardScaler, PowerTransformer
import matplotlib.pyplot as plt
import seaborn as sns

print("=" * 80)
print("ADVANCED FEATURE NORMALIZATION - FIXING DOMINANCE ISSUES")
print("=" * 80)

# =============================================================================
# SETUP
# =============================================================================
project_root = Path("/workspaces/instacart-customer-clustering")
features_dir = project_root / "artifacts" / "features"
models_dir = project_root / "artifacts" / "models"
figures_dir = project_root / "artifacts" / "figures"

# =============================================================================
# 1. LOAD FEATURES
# =============================================================================
print("\n[1] Loading features...")

user_features = pd.read_csv(features_dir / "user_features.csv")
user_features_scaled = pd.read_csv(features_dir / "user_features_scaled.csv")

feature_cols = [col for col in user_features.columns if col != "user_id"]
print(f"  Features: {len(feature_cols)}")

# =============================================================================
# 2. ANALYZE CURRENT SCALING ISSUES
# =============================================================================
print("\n[2] Analyzing scaling quality...")

scaled_variance = user_features_scaled[feature_cols].var()
scaled_mean = user_features_scaled[feature_cols].mean()
scaled_std = user_features_scaled[feature_cols].std()

print(f"\n  Current scaled statistics:")
print(f"    Mean range: [{scaled_mean.min():.4f}, {scaled_mean.max():.4f}]")
print(f"    Std range: [{scaled_std.min():.4f}, {scaled_std.max():.4f}]")
print(f"    Variance range: [{scaled_variance.min():.4f}, {scaled_variance.max():.4f}]")

# Check for dominance
variance_ratio = scaled_variance.max() / scaled_variance.min()
print(f"\n  ⚠️  Variance ratio (max/min): {variance_ratio:.2f}x")

if variance_ratio > 3:
    print(f"  ⚠️  HIGH VARIANCE INEQUALITY! Features will dominate clustering.")
    print(f"  ⚠️  Recommended: < 3x, Current: {variance_ratio:.2f}x")

# Identify problematic features
high_var_features = scaled_variance[scaled_variance > scaled_variance.median() * 2]
print(f"\n  Features with excessive variance (>{scaled_variance.median()*2:.2f}):")
for feat, var in high_var_features.items():
    print(f"    {feat}: {var:.4f}")

# =============================================================================
# 3. APPLY VARIANCE STABILIZATION
# =============================================================================
print("\n[3] Applying variance stabilization transformations...")

# Strategy: Multi-step normalization
# Step 1: Log transform for heavy-tailed features
# Step 2: Power transform for skewed features  
# Step 3: Standard scaling to unit variance

# Identify features needing log transform (count features)
log_transform_features = [
    'total_product_instances',
    'total_orders',
    'unique_products'
]

# Identify features needing power transform (ratio features)
power_transform_features = [col for col in feature_cols 
                             if col.startswith('aisle_') or 
                             col in ['organic_ratio', 'fresh_ratio']]

# Copy original features
user_features_normalized = user_features[["user_id"]].copy()

print(f"\n  → Applying transformations:")
print(f"    Log transform: {len(log_transform_features)} features")
print(f"    Power transform: {len(power_transform_features)} features")
print(f"    Standard scaling: {len(feature_cols)} features (final)")

# =============================================================================
# TRANSFORMATION PIPELINE
# =============================================================================

transformed_features = user_features[feature_cols].copy()

# Step 1: Log transform count features (stabilizes variance)
for feat in log_transform_features:
    if feat in transformed_features.columns:
        # Add 1 to avoid log(0)
        transformed_features[feat] = np.log1p(transformed_features[feat])
        print(f"      Log-transformed: {feat}")

# Step 2: Power transform ratio/aisle features (fixes skewness)
pt = PowerTransformer(method='yeo-johnson', standardize=False)
for feat in power_transform_features:
    if feat in transformed_features.columns:
        # Reshape for sklearn
        values = transformed_features[[feat]].values
        transformed_features[feat] = pt.fit_transform(values).flatten()

print(f"      Power-transformed: {len(power_transform_features)} features")

# Step 3: StandardScaler to unit variance (critical!)
print(f"\n  → Applying StandardScaler for equal variance...")
scaler_std = StandardScaler()
scaled_values = scaler_std.fit_transform(transformed_features)

normalized_df = pd.DataFrame(
    scaled_values,
    columns=feature_cols,
    index=user_features.index
)

user_features_normalized = pd.concat([user_features_normalized, normalized_df], axis=1)

# =============================================================================
# 4. VERIFY IMPROVEMENTS
# =============================================================================
print("\n[4] Verifying normalization quality...")

new_variance = normalized_df.var()
new_mean = normalized_df.mean()
new_std = normalized_df.std()

print(f"\n  NEW scaled statistics:")
print(f"    Mean range: [{new_mean.min():.4f}, {new_mean.max():.4f}]")
print(f"    Std range: [{new_std.min():.4f}, {new_std.max():.4f}]")
print(f"    Variance range: [{new_variance.min():.4f}, {new_variance.max():.4f}]")

new_variance_ratio = new_variance.max() / new_variance.min()
print(f"\n  ✅ NEW variance ratio: {new_variance_ratio:.2f}x")
print(f"  ✅ Improvement: {variance_ratio:.2f}x → {new_variance_ratio:.2f}x")

if new_variance_ratio < 3:
    print(f"  ✅ GOOD! Variance inequality is now acceptable.")
else:
    print(f"  ⚠️  Still high. Consider removing outliers or additional transforms.")

# =============================================================================
# 5. SIDE-BY-SIDE COMPARISON
# =============================================================================
print("\n[5] Creating comparison visualizations...")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Before
axes[0].bar(range(len(scaled_variance)), scaled_variance.sort_values(ascending=False))
axes[0].set_title("BEFORE: Feature Variance (Unequal)", fontweight='bold', fontsize=14)
axes[0].set_xlabel("Feature Rank")
axes[0].set_ylabel("Variance")
axes[0].axhline(y=1.0, color='r', linestyle='--', label='Target Variance')
axes[0].legend()
axes[0].grid(alpha=0.3)

# After
axes[1].bar(range(len(new_variance)), new_variance.sort_values(ascending=False))
axes[1].set_title("AFTER: Feature Variance (Normalized)", fontweight='bold', fontsize=14)
axes[1].set_xlabel("Feature Rank")
axes[1].set_ylabel("Variance")
axes[1].axhline(y=1.0, color='r', linestyle='--', label='Target Variance')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
variance_comparison_path = figures_dir / "variance_normalization_comparison.png"
plt.savefig(variance_comparison_path, dpi=300, bbox_inches='tight')
print(f"  ✓ Saved: {variance_comparison_path.name}")
plt.close()

# Feature-wise comparison
comparison_df = pd.DataFrame({
    'feature': feature_cols,
    'variance_before': scaled_variance.values,
    'variance_after': new_variance.values,
    'improvement': ((scaled_variance.values - new_variance.values) / scaled_variance.values * 100)
})

comparison_df = comparison_df.sort_values('variance_before', ascending=False)

fig, ax = plt.subplots(figsize=(14, 8))
x = np.arange(len(comparison_df))
width = 0.35

ax.barh(x - width/2, comparison_df['variance_before'], width, 
        label='Before', alpha=0.8, color='coral')
ax.barh(x + width/2, comparison_df['variance_after'], width, 
        label='After', alpha=0.8, color='skyblue')

ax.set_yticks(x)
ax.set_yticklabels(comparison_df['feature'], fontsize=8)
ax.set_xlabel('Variance', fontweight='bold')
ax.set_title('Feature Variance: Before vs After Normalization', 
             fontweight='bold', fontsize=14)
ax.legend()
ax.grid(alpha=0.3, axis='x')
ax.axvline(x=1.0, color='green', linestyle='--', linewidth=2, label='Target')

plt.tight_layout()
detailed_comparison_path = figures_dir / "feature_variance_detailed_comparison.png"
plt.savefig(detailed_comparison_path, dpi=300, bbox_inches='tight')
print(f"  ✓ Saved: {detailed_comparison_path.name}")
plt.close()

# =============================================================================
# 6. CORRELATION MATRIX COMPARISON
# =============================================================================
print("\n[6] Checking feature correlations...")

# Identify highly correlated features (redundancy check)
corr_matrix = normalized_df.corr().abs()

# Find pairs with correlation > 0.8
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if corr_matrix.iloc[i, j] > 0.8:
            high_corr_pairs.append({
                'feature1': corr_matrix.columns[i],
                'feature2': corr_matrix.columns[j],
                'correlation': corr_matrix.iloc[i, j]
            })

if len(high_corr_pairs) > 0:
    print(f"\n  ⚠️  Found {len(high_corr_pairs)} highly correlated feature pairs (>0.8):")
    for pair in high_corr_pairs[:5]:  # Show top 5
        print(f"    {pair['feature1']} ↔ {pair['feature2']}: {pair['correlation']:.3f}")
    print(f"  💡 Consider removing one from each pair to reduce redundancy.")
else:
    print(f"  ✅ No highly correlated features found.")

# =============================================================================
# 7. SAVE NORMALIZED FEATURES
# =============================================================================
print("\n[7] Saving normalized features...")

# Save final features
normalized_path = features_dir / "user_features_normalized.csv"
user_features_normalized.to_csv(normalized_path, index=False)
print(f"  ✓ Normalized features: {normalized_path.name}")

# Save transformation pipeline
pipeline = {
    'log_transform_features': log_transform_features,
    'power_transform_features': power_transform_features,
    'standard_scaler': scaler_std
}

pipeline_path = models_dir / "normalization_pipeline.pkl"
with open(pipeline_path, 'wb') as f:
    pickle.dump(pipeline, f)
print(f"  ✓ Pipeline: {pipeline_path.name}")

# Save comparison report
comparison_df.to_csv(features_dir / "variance_comparison_report.csv", index=False)
print(f"  ✓ Report: variance_comparison_report.csv")

# =============================================================================
# 8. FINAL RECOMMENDATIONS
# =============================================================================
print("\n" + "=" * 80)
print("✅ FEATURE NORMALIZATION COMPLETE!")
print("=" * 80)

print(f"\n📊 Summary:")
print(f"  Variance ratio improvement: {variance_ratio:.2f}x → {new_variance_ratio:.2f}x")
print(f"  Mean centering: {abs(new_mean).max():.4f} (closer to 0 is better)")
print(f"  Std deviation: {new_std.min():.4f} - {new_std.max():.4f}")

print(f"\n📁 Files Created:")
print(f"  {normalized_path}")
print(f"  {pipeline_path}")
print(f"  {variance_comparison_path}")
print(f"  {detailed_comparison_path}")

print(f"\n🎯 RECOMMENDATION:")
if new_variance_ratio < 2:
    print(f"  ✅ USE: user_features_normalized.csv for clustering")
    print(f"  ✅ Features are well-balanced and ready for K-Means/DBSCAN")
elif new_variance_ratio < 3:
    print(f"  ⚠️  USE: user_features_normalized.csv (acceptable)")
    print(f"  💡 Consider testing both normalized and original scaled versions")
else:
    print(f"  ⚠️  Variance still high. Next steps:")
    print(f"     1. Remove outliers (filter top 1% of extreme values)")
    print(f"     2. Try PCA (reduces to uncorrelated components)")
    print(f"     3. Use distance metrics robust to scale (Mahalanobis)")

print(f"\n💡 For K-Means: Use user_features_normalized.csv")
print(f"💡 For DBSCAN: Test both (DBSCAN is more scale-sensitive)")
print(f"💡 For Hierarchical: Normalized version is preferred")

# =============================================================================
# 9. OPTIONAL: REMOVE REDUNDANT FEATURES
# =============================================================================
if len(high_corr_pairs) > 0:
    print(f"\n[OPTIONAL] Removing redundant features...")
    
    # Identify features to drop (keep first of each pair)
    features_to_drop = set()
    for pair in high_corr_pairs:
        # Keep feature with higher variance
        var1 = new_variance[pair['feature1']]
        var2 = new_variance[pair['feature2']]
        
        if var1 < var2:
            features_to_drop.add(pair['feature1'])
        else:
            features_to_drop.add(pair['feature2'])
    
    print(f"  Features to drop (redundant): {features_to_drop}")
    
    # Create reduced feature set
    features_to_keep = [col for col in feature_cols if col not in features_to_drop]
    
    user_features_reduced = user_features_normalized[["user_id"] + features_to_keep].copy()
    
    reduced_path = features_dir / "user_features_reduced.csv"
    user_features_reduced.to_csv(reduced_path, index=False)
    
    print(f"  ✓ Reduced features: {reduced_path.name}")
    print(f"  ✓ Reduced from {len(feature_cols)} → {len(features_to_keep)} features")

ADVANCED FEATURE NORMALIZATION - FIXING DOMINANCE ISSUES

[1] Loading features...
  Features: 30

[2] Analyzing scaling quality...

  Current scaled statistics:
    Mean range: [-0.1131, 0.5934]
    Std range: [0.5321, 1.2847]
    Variance range: [0.2832, 1.6504]

  ⚠️  Variance ratio (max/min): 5.83x
  ⚠️  HIGH VARIANCE INEQUALITY! Features will dominate clustering.
  ⚠️  Recommended: < 3x, Current: 5.83x

  Features with excessive variance (>1.08):
    total_orders: 1.2901
    total_product_instances: 1.6504

[3] Applying variance stabilization transformations...

  → Applying transformations:
    Log transform: 3 features
    Power transform: 12 features
    Standard scaling: 30 features (final)
      Log-transformed: total_product_instances
      Log-transformed: total_orders
      Log-transformed: unique_products
      Power-transformed: 12 features

  → Applying StandardScaler for equal variance...

[4] Verifying normalization quality...

  NEW scaled statistics:
    Mean range: 